## Smooth Power Law Examples

Below are a few examples of the smoothly broken power law rating curve implementation.

In [ ]:
from numpy.typing import ArrayLike
import numpy as np
import matplotlib.pyplot as plt
import arviz as az

import sys
if "../.." not in sys.path:
    sys.path.insert(0, "../..")

Equation for a smoothly broken power law with $n$ breaks is:

\begin{equation}
    f(x) = Ax^{\alpha_0} {\displaystyle \prod^{n}_{i=1}} \Bigg(1 + \bigg(\frac{x}{x_i}\bigg)^{1/\Delta}\Bigg)^{(\alpha_i - \alpha_{i-1})\Delta},
\end{equation}

where $A$ is the scaling amplitude, $\alpha_0$ is the power law slope before the first breakpoint ($x_1$), $\alpha_i$ are the power law slopes between breakpoints $x_{i}$ and $x_{i+1}$ with $i$ being the $i$th breakpoint and $i < n$, $\alpha_n$ is the power law slope after the last breakpoint ($x_n$), and $\Delta$ is the parameter controlling the smoothness of the break. Smaller (non-negative) values of $\Delta$ yield a sharper break, and larger values yield a smoother break.

This equation was adapted from Equation 1 in [Caballero et al. 2023](https://arxiv.org/pdf/2210.14891.pdf).

In log-space:

\begin{equation}
    \log(f(x)) = \log(A) + \alpha_0 \log(x) + {\displaystyle \sum^{n}_{i=1}} (\alpha_i - \alpha_{i-1})\Delta \Bigg(1 + \bigg(\frac{x}{x_i}\bigg)^{1/\Delta}\Bigg),
\end{equation}

With $x_0$ offset:

\begin{equation}
    \log(f(x)) = \log(A) + \alpha_0 \log(x - x_0) + {\displaystyle \sum^{n}_{i=1}} (\alpha_i - \alpha_{i-1})\Delta \Bigg(1 + \bigg(\frac{x - x_0}{x_i}\bigg)^{1/\Delta}\Bigg),
\end{equation}

In [ ]:
def SmoothlyBrokenPowerLaw(x: ArrayLike, amplitude: float, breaks: ArrayLike, alphas: ArrayLike, delta: float) -> np.ndarray:
    breakpoint_array = np.expand_dims(breaks, 0)
    alphas_array = np.array(alphas)
    alphas_diff_array = np.expand_dims(alphas_array[1:] - alphas_array[:-1], 0)
    x_array = np.expand_dims(x, 1)
    prod_array = (1 + ((x_array - breakpoint_array[0, 0])/breakpoint_array[:, 1:]) ** (1/delta)) ** (alphas_diff_array * delta)
    fx = amplitude * (x - breakpoint_array[0, 0]) ** alphas[0] * np.prod(prod_array, axis=1)
    return fx

Now that we have created an equation function, let's generate some simulated data.

In [ ]:
h = np.geomspace(3, 20, 200)
a = 0.01
delta = 0.05
breaks = [0, 7.]
alphas = [0.9, 2]
q = SmoothlyBrokenPowerLaw(h, a, breaks, alphas, delta)
# Add some random noise in log space
q = np.exp(np.log(q) + np.random.randn(len(q)) * 0.01)

fig, ax = plt.subplots()
ax.loglog(q, h, marker='o', linestyle='')
ax.set(xlabel='Discharge', ylabel='Stage')
plt.show

Then fit the simulated data and plot the fits.

In [ ]:
from ratingcurve.experimental_ratings import SmoothlyBrokenPowerLawRating

# initialize the model
# Default model configurations are okay, as we want uniform priors and 2 segments
smoothpowerrating = SmoothlyBrokenPowerLawRating()
smoothpowerrating.fit(h, q, method='nuts')
data = smoothpowerrating.table()

In [ ]:
fig, ax = plt.subplots()
ax.loglog(q, h, marker='o', linestyle='', label='Data')
ax.loglog(data['discharge'], data['stage'], label='Model Fit')
ax.set(xlabel='Discharge', ylabel='Stage')
plt.legend()
plt.show

In [ ]:
smoothpowerrating.plot()

In [ ]:
smoothpowerrating.plot_residuals()

So, the fit looks good compared to the data. Let's also check the derived values.

In [ ]:
print('Break point expected values: '+str(breaks)+', Derived values: '
      +str(smoothpowerrating.idata.posterior.hs.mean(dim=['draw', 'chain']).values.squeeze())+' +/- '+str(smoothpowerrating.idata.posterior.hs.std(dim=['draw', 'chain']).values.squeeze()))
print('Smoothness parameter expected values: '+str(delta)+', Derived values: '
      +str(smoothpowerrating.idata.posterior.delta.mean(dim=['draw', 'chain']).values.squeeze())+' +/- '+str(smoothpowerrating.idata.posterior.delta.std(dim=['draw', 'chain']).values.squeeze()))

Now let's test the model on some real data to see how it compares on something not generated by itself.

In [ ]:
from ratingcurve import data

# load tutorial data
df = data.load('green channel')

fig, ax = plt.subplots()
ax.errorbar(df['q'], df['stage'], xerr=df['q_sigma'], marker='o', linestyle='', fillstyle='none')
ax.set(xlabel='Discharge', ylabel='Stage', xscale='log', yscale='log')
plt.show

In [ ]:
# initialize the model
# Again we only need 2 segments (the default)
smoothpowerrating_gc = SmoothlyBrokenPowerLawRating()
smoothpowerrating_gc.fit(df['stage'], df['q'], q_sigma=df['q_sigma'], method='nuts')
data_gc = smoothpowerrating_gc.table()

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(df['q'], df['stage'], xerr=df['q_sigma'], marker='o', linestyle='', fillstyle='none', label='Data')
ax.loglog(data_gc['discharge'], data_gc['stage'], label='Smooth Model Fit')
ax.set(xlabel='Discharge', ylabel='Stage', xscale='log', yscale='log')
plt.legend()
plt.show

In [ ]:
smoothpowerrating_gc.summary(var_names=['hs', 'delta'])

In [ ]:
smoothpowerrating_gc.plot()